# Занятие 9. Декораторы
### Практикум ММП ВМК МГУ
Васильев Руслан, 2021
***

## Введение

In [1]:
def make_bold(func):
    def wrapper():
        return '<b>' + func() + '</b>'
    return wrapper

In [2]:
@make_bold
def say():
    return 'something'

In [3]:
say()

'<b>something</b>'

In [4]:
def make_bold(func):
    def wrapper(*args, **kwargs):
        return '<b>' + func(*args, **kwargs) + '</b>'
    return wrapper
 
def make_italic(func):
    def wrapper(*args, **kwargs):
        return '<i>' + func(*args, **kwargs) + '</i>'
    return wrapper

In [5]:
@make_bold
@make_italic
def say(s):
    return s

In [6]:
say('123')

'<b><i>123</i></b>'

<div class="alert alert-block alert-success">
<b>Синтаксический сахар</b> Декораторы делают код более читаемым: сразу понимаем, что функция задекорирована.
</div>

In [36]:
def say(s):
    return s
say_bold_italic = make_bold(make_italic(say))
say_bold_italic('123')

'<b><i>123</i></b>'

## Что происходит внутри

In [37]:
def make_bold_debug(func):
    print('Start decorating...')
    def wrapper_my(*args, **kwargs):
        '''Whis is wrapper'''
        output = '<b>' + func(*args, **kwargs) + '</b>'
        print('Inside `wrapper`')
        return output
    print('Finish decorating.')
    return wrapper_my

@make_bold_debug
def say_debug(s):
    '''This is say debug'''
    print('Inside `say_debug`')
    return s

Start decorating...
Finish decorating.


In [38]:
say_debug('123')

Inside `say_debug`
Inside `wrapper`


'<b>123</b>'

## Изменение параметров декоратора

<div class="alert alert-block alert-danger">
<b>Проблема:</b> параметры задекорированной функции соответствуют обертке.
</div>

In [39]:
from functools import wraps

def make_bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        '''Make text bold'''
        return '<b>' + func(*args, **kwargs) + '</b>'
    return wrapper

def make_italic(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        '''Make text italic'''
        return '<i>' + func(*args, **kwargs) + '</i>'
    return wrapper

@make_bold
@make_italic
def say(s):
    '''Say something'''
    return s

Что делает `funtools.wraps` внутри? Вызывает [funtools.update_wrapper](https://docs.python.org/3/library/functools.html#functools.update_wrapper).

In [40]:
from functools import update_wrapper

def my_decorator(f):
    def my_wraps(wrapper):
        return update_wrapper(wrapper, f)
    @my_wraps
    def wrapper(*args, **kwargs):
        '''Inner function'''
        return f(*args, **kwargs)
    return wrapper

In [41]:
@my_decorator
def f():
    pass

In [42]:
help(f)

Help on function f in module __main__:

f()



In [43]:
print(f)
print(f.__wrapped__)

<function f at 0x7f10a8520550>
<function f at 0x7f10a8520670>


Что делает `functools.update_wrapper`? Обновляет/переписывает атрибуты:

In [44]:
def my_decorator(f):
    def wrapper(*args, **kwargs):
        '''Inner function'''
        return f(*args, **kwargs)
    wrapper.__module__ = f.__module__
    wrapper.__name__ = f.__name__
    wrapper.__doc__ = f.__doc__
    wrapper.__wrapped__ = f
    return wrapper

In [45]:
@my_decorator
def f():
    pass

In [47]:
help(f)

Help on function f in module __main__:

f()



## Декораторы с аргументами

### Quick start

In [48]:
times = 4

def asterisk_decorator(f):
    def wrapper(*args, **kwargs):
        prefix = suffix = '*' * times
        f_result = f(*args, **kwargs)
        return prefix + f_result + suffix
    return wrapper

In [49]:
@asterisk_decorator
def say_hello():
    return 'Hello!'

In [50]:
say_hello()

'****Hello!****'

<div class="alert alert-block alert-info">
<b>Хочется:</b> кастомизировать не только то, что мы оборачиваем, но и сами обертки.
</div>

In [51]:
def add_asterisks(times):
    def asterisk_decorator(f):
        def wrapper(*args, **kwargs):
            prefix = suffix = '*' * times
            f_result = f(*args, **kwargs)
            return prefix + f_result + suffix
        return wrapper
    return asterisk_decorator

In [52]:
@add_asterisks(10)
def say_hello():
    return 'Hello!'

In [53]:
say_hello()

'**********Hello!**********'

<div class="alert alert-block alert-warning">
<b>Важно:</b> Декоратор вызывается <b>ровно один раз</b>. После импорта функции уже задекорированы.
</div>

### Значения по умолчанию

In [54]:
def add_emoji(emoji, times):
    def emoji_decorator(f):
        def wrapper(*args, **kwargs):
            suffix = ' ' + f'{emoji} ' * times
            f_result = f(*args, **kwargs)
            return f_result + suffix
        return wrapper
    return emoji_decorator

In [55]:
@add_emoji('🤡', 3)
def say_bye():
    return 'Poka'

In [56]:
say_bye()

'Poka 🤡 🤡 🤡 '

In [57]:
def add_emoji(emoji='🙄', times=3):
    def emoji_decorator(f):
        def wrapper(*args, **kwargs):
            suffix = ' ' + f'{emoji} ' * times
            f_result = f(*args, **kwargs)
            return f_result + suffix
        return wrapper
    return emoji_decorator

In [58]:
@add_emoji
def what_is_your_name():
    return '성기훈'

In [59]:
what_is_your_name()

TypeError: emoji_decorator() missing 1 required positional argument: 'f'

In [60]:
@add_emoji(emoji=')')
def what_is_your_name():
    return '성기훈'

In [61]:
what_is_your_name()

'성기훈 ) ) ) '

In [62]:
def add_emoji_debug(emoji='🙄', times=3):

    print('Inside decorator fabric')
    print(f'{emoji=}, {times=}')

    def emoji_decorator_debug(f):
        def wrapper(*args, **kwargs):
            suffix = ' ' + f'{emoji} ' * times
            f_result = f(*args, **kwargs)
            return f_result + suffix
        return wrapper

    return emoji_decorator_debug

@add_emoji_debug
def what_is_your_name():
    return '성기훈'

Inside decorator fabric
emoji=<function what_is_your_name at 0x7f10a8520160>, times=3


### Декораторы для декораторов и как избавиться от тройной вложенности

In [63]:
def decorator_with_params(decorator):
    '''Декоратор для декораторов'''

    def make_decorator(*args, **kwargs):

        def decorator_wrapper(f):
            return decorator(f, *args, **kwargs)

        return decorator_wrapper

    return make_decorator

In [64]:
@decorator_with_params
def add_emoji(f, emoji='🙄', times=3):
    def wrapper(*args, **kwargs):
        suffix = ' ' + f'{emoji} ' * times
        f_result = f(*args, **kwargs)
        return f_result + suffix
    return wrapper

In [65]:
@add_emoji()
def what_is_your_name():
    return '성기훈'

In [66]:
what_is_your_name()

'성기훈 🙄 🙄 🙄 '

In [67]:
@add_emoji(emoji='😷')
def what_is_your_name():
    return '성기훈'

In [68]:
what_is_your_name()

'성기훈 😷 😷 😷 '

### Обработка обоих сценариев

<div class="alert alert-block alert-info">
    <b>Костыли:</b> Чтобы каждый раз не писать костыль <b>decorator()</b>, заранее напишем костыль внутри декоратора декораторов.
</div>

In [69]:
def cool_decorator_with_params(decorator):
    '''Крутой декоратор для декораторов'''

    def make_decorator(*args, **kwargs):

        if len(args) == 1 and kwargs == {} and callable(args[0]):
            return decorator(args[0])

        else:
            def decorator_wrapper(f):
                return decorator(f, *args, **kwargs)
            return decorator_wrapper

    return make_decorator

In [70]:
@cool_decorator_with_params
def add_emoji(f, emoji='☕', times=3):
    def wrapper(*args, **kwargs):
        suffix = ' ' + f'{emoji} ' * times
        f_result = f(*args, **kwargs)
        return f_result + suffix
    return wrapper

In [71]:
@add_emoji
def London_is_the():
    return 'capital of Great Britain'

In [72]:
London_is_the()

'capital of Great Britain ☕ ☕ ☕ '

## Реализация декоратора с помощью класса

<div class="alert alert-block alert-success">
<b>Хмм,</b> декоратор — callable объект. Чтобы экземпляр класса можно было использовать как декоратор, нужно определить метод <b>__call__</b>.
</div>

In [73]:
from time import time

class Timer:
    def __init__(self, f):
        self.f = f

    def __call__(self, *args, **kwargs):
        start = time()
        result = self.f(*args, **kwargs)
        duration = time() - start
        print(f'{self.f.__name__} duration: {duration:3f} seconds')
        return result

In [74]:
@Timer
def slow_funtion(n=7):
    return sum(range(10 ** n))

In [75]:
slow_funtion(8)

slow_funtion duration: 1.725039 seconds


4999999950000000

## Декораторы для класса

In [76]:
def add_cpu(target_class):
    print('Inside add cpu')
    target_class.device = 'cpu'
    return target_class

In [77]:
@add_cpu
class MachineLearning:
    def __init__(self, n_layers=96):
        print('__init__')
        self.n_layers=96

    def train(self):
        pass

Inside add cpu


In [78]:
model = MachineLearning()
model.device

__init__


'cpu'

In [79]:
def add_device(device):
    print('add_device')
    def wrapper(target_class):
        print('wrapper')
        target_class.device = device
        return target_class
    return wrapper

In [80]:
@add_device('cuda')
class MachineLearning:
    def __init__(self, n_layers=96):
        self.n_layers=96

    def train(self):
        pass

add_device
wrapper


In [81]:
model = MachineLearning()
model.device

'cuda'

## Готовые декораторы

### Кэширование и мемоизация

In [82]:
from functools import lru_cache

In [92]:
@lru_cache(maxsize=1000)
def factorial(n):
    if n == 0:
        return 1
    elif n > 0:
        return n * factorial(n - 1)
    else:
        raise ValueError('Factorial argument should be non-negative integer')

In [93]:
%%time
_ = factorial(1000)

CPU times: user 1.5 ms, sys: 0 ns, total: 1.5 ms
Wall time: 1.51 ms


In [94]:
%%time
_ = factorial(1000)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


* В простейших случаях можно обойтись [functools.cache](https://docs.python.org/3/library/functools.html#functools.cache) — нет лимитов
* Библиотека для мемоизации (еще больше декораторов): https://github.com/tkem/cachetools

In [95]:
import numpy as np
from joblib import Memory

cachedir = './here'
memory = Memory(cachedir, verbose=0)

@memory.cache
def pow2(X):
    return X ** 2

In [96]:
pow2(np.arange(10))

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [98]:
!ls ./here

joblib


## Варнгинги

In [99]:
from warnings import warn

In [100]:
def deprecated(f):
    message = f.__name__ + ' is deprecated and will be removed in future version'
    warn(message)
    return f

In [101]:
@deprecated
def mda():
    pass

/tmp/ipykernel_26706/638169632.py:3: UserWarning: mda is deprecated and will be removed in future version
  warn(message)


## Декораторы для методов класса

<div class="alert alert-block alert-success">
Наиболее часто встречающиеся декораторы для методов класса:
 <ul>
  <li><b>@property</b> для вычисляемых атрибутов;</li>
  <li><b>@staticmethod</b> для превращения метода в статическую функцию (аналог в C++), при вызове неявные первый аргумент не передается;</li>
  <li><b>@classmethod</b>  для передачи первым аргументов всего класса, а не экземпляра.</li>
</ul> 
</div>

In [102]:
class Date(object):
    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year
    
    @property
    def favourite_year(self):
        return self.year

    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date = cls(day, month, year)
        return date

    @staticmethod
    def is_date_2021(date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        return year == 2021

    def __repr__(self):
        return f'Date({self.day}, {self.month}, {self.year})'

In [103]:
date = Date(25, 10, 2021)
date

Date(25, 10, 2021)

In [104]:
Date.from_string('25-11-2021')

Date(25, 11, 2021)

In [105]:
date.favourite_year

2021

In [106]:
date.is_date_2021()

TypeError: is_date_2021() missing 1 required positional argument: 'date_as_string'

In [107]:
Date.is_date_2021('31-10-2021')

True

## Dataclass

In [108]:
from dataclasses import dataclass, field

In [109]:
@dataclass
class DataLoader:
    dataset_path: str
    batch_size: int
    num_workers: int
    drop_last: bool = False
    output_paths: list[str] = field(default_factory=lambda: ['./'])
        
    def __post_init__(self):
        self.seed = 0

In [110]:
dl = DataLoader('path/to/dataset', 32, 2, drop_last=True)

In [111]:
dl.output_paths

['./']

In [112]:
dl.seed

0

## Несортированные примеры

### Отладка: выводим размеры тензора

In [113]:
import numpy as np

def print_shapes(f):
    @wraps(f)
    def wrapper(**kwargs):
        print('*** Shapes ***')
        for name, obj in kwargs.items():
            if isinstance(obj, np.ndarray):
                print(f'Input {name}: {obj.shape}')
        result = f(**kwargs)
        if isinstance(result, np.ndarray):
            print(f'Output: {result.shape}')
        print('***')
        return result
    return wrapper

In [114]:
@print_shapes
def LinearLayer(*, X, W, b):
    return X @ W + b

```python
def func(param1, param2, /, param3, *, param4, param5):
    pass
```
* `param1`, `param2`: positional-only
* `param3`: positional or keyword
* `param4`, `param5`: keyword-only

Подробнее (Python 3.8): [документация](https://docs.python.org/3/reference/compound_stmts.html#function-definitions).

In [115]:
_ = LinearLayer(
    X=np.random.randn(20, 10, 30),
    W=np.random.randn(30, 100),
    b=np.random.randn(100),
)

*** Shapes ***
Input X: (20, 10, 30)
Input W: (30, 100)
Input b: (100,)
Output: (20, 10, 100)
***


* Можно повозиться с позиционными аргументами с помощью [inspect.getfullargspec](https://docs.python.org/3/library/inspect.html#inspect.getfullargspec)
* Можно (и даже лучше) использовать [type hints](https://docs.python.org/3/library/typing.html).

In [116]:
@print_shapes
def LinearLayer(
    dummy_param,
    X: np.ndarray,
    W: np.ndarray,
    b: np.ndarray,
    scale: float = 2.0,
    hello_who: str = 'world?'
) -> np.ndarray:

    print(f'Hello, {hello_who}')

    X_scaled = X / scale
    Y_scaled = X_scaled @ W
    Y = Y * scale
    output = Y + b

    return output

In [117]:
LinearLayer.__annotations__

{'X': numpy.ndarray,
 'W': numpy.ndarray,
 'b': numpy.ndarray,
 'scale': float,
 'hello_who': str,
 'return': numpy.ndarray}

In [118]:
from typing import get_type_hints

get_type_hints(LinearLayer)

{'X': numpy.ndarray,
 'W': numpy.ndarray,
 'b': numpy.ndarray,
 'scale': float,
 'hello_who': str,
 'return': numpy.ndarray}

In [119]:
from inspect import getfullargspec

getfullargspec(LinearLayer)

FullArgSpec(args=[], varargs=None, varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={'return': <class 'numpy.ndarray'>})

# Links
* https://github.com/mmp-practicum-team/mmp_practicum_fall_2020/blob/master/09_decorators/decorators.pdf
* https://habr.com/ru/post/141411/
* https://habr.com/ru/post/141501/
* https://www.learnpython.org/en/Decorators
* https://wiki.python.org/moin/FunctionWrappers
* https://www.kite.com/python/answers/how-to-decorate-a-class-in-python
* https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner
* http://uneex.ru/LecturesCMC/PythonIntro2020/11_MiscOOP
* https://www.kite.com/python/answers/how-to-decorate-a-class-in-python
* https://docs.python.org/3/library/dataclasses.html